In [1]:
from file_manager import load_system, select_atoms_by_Z, csv_export_atomic_system
from information_measures import *
import wave_function_constructor as wfc
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import numpy as np
BASE_PATH = "COEFS" #Leo desde 

neutral_atom = load_system(0, BASE_PATH) 
atoms_selected = select_atoms_by_Z(neutral_atom, 'all') #Seleccionamos todos los átomos neutros

atoms_selected = atoms_selected[0:50]

In [ ]:
if __name__ == "__main__":
    import time
    
    from concurrent.futures import ThreadPoolExecutor, as_completed
    from tqdm import tqdm

    # Asegúrate de tener estas clases importadas
    # from your_module import ShannonEntropy, EntropicMoment, RenyiEntropy

    def compute_entropy(name, cls, kwargs):
        """Ejecuta una tarea de cómputo de entropía y devuelve su nombre y resultado."""
        return name, cls(**kwargs)

    start_time = time.time()

    qs_values = [num / 4 for num in range(2, 13)]
    alpha_values = [num / 5 for num in range(1, 11)]

    tasks = []

    # Shannon Entropy
    for atom in atoms_selected:
        tasks += [
            ("Shannon - posicion", ShannonEntropy, {"atoms": atom, "space": "position"}),
            ("Shannon - momento", ShannonEntropy, {"atoms": atom, "space": "momentum"})
        ]

        # EntropicMoment: una tarea por valor de qs
        for q in qs_values:
            tasks += [
                (f"EntropicMoment - posicion q={q:.3f} - atom: {atom.Symbol}", EntropicMoment, {"atoms": atom, "qs": [q], "space": "position"}),
                (f"EntropicMoment - momento q={q:.3f} - atom: {atom.Symbol}", EntropicMoment, {"atoms": atom, "qs": [q], "space": "momentum"})
            ]

        # Renyi Entropy: una tarea por valor de alpha
        for a in alpha_values:
            tasks += [
                (f"Renyi - posicion alpha={a:.3f}- atom: {atom.Symbol}", RenyiEntropy, {"atoms": atom, "alpha": [a], "space": "position"}),
                (f"Renyi - momento alpha={a:.3f}- atom: {atom.Symbol}", RenyiEntropy, {"atoms": atom, "alpha": [a], "space": "momentum"})
            ]
    for atom in atoms_selected:
        tasks += [("Shannon - posicion (2e)- atom: {atom.Symbol}", ShannonEntropy, {"atoms": atom, "space": "position", "two_electron_density": True}),
            ("Shannon - momento (2e)- atom: {atom.Symbol}", ShannonEntropy, {"atoms": atom, "space": "momentum", "two_electron_density": True})]
        
        for q in qs_values:
            tasks += [
                (f"EntropicMoment - posicion (2e) q={q:.3f}- atom: {atom.Symbol}", EntropicMoment, {"atoms": atom, "qs": [q], "space": "position", "two_electron_density": True}),
                (f"EntropicMoment - momento (2e) q={q:.3f}- atom: {atom.Symbol}", EntropicMoment, {"atoms": atom, "qs": [q], "space": "momentum", "two_electron_density": True})
            ]

        for a in alpha_values:
            tasks += [
                (f"Renyi - posicion (2e) alpha={a:.3f}- atom: {atom.Symbol}", RenyiEntropy, {"atoms": atom, "alpha": [a], "space": "position", "two_electron_density": True}),
                (f"Renyi - momento (2e) alpha={a:.3f}- atom: {atom.Symbol}", RenyiEntropy, {"atoms": atom, "alpha": [a], "space": "momentum", "two_electron_density": True})
            ]

    results = {}
    in_progress = set()

    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_name = {
            executor.submit(compute_entropy, name, cls, kwargs): name
            for name, cls, kwargs in tasks
        }

        with tqdm(total=len(future_to_name), desc="Calculando entropías") as pbar:
            for future in as_completed(future_to_name):
                name = future_to_name[future]
                in_progress.add(name)
                try:
                    task_name, result = future.result()
                    results[task_name] = result
                    tqdm.write(f"✅ Completado: {task_name}")
                except Exception as e:
                    tqdm.write(f"❌ Error en: {name} — {e}")
                pbar.update(1)

    end_time = time.time()
    print(f"\nTiempo total de cómputo: {end_time - start_time:.2f} segundos")



c:\Users\Ivan_lg\Desktop\Doctorado\SegundoAnio\TFGMariadelMar\Calculos\hartreeFock\information_measures.py:538: UserWarning: Space provided as string. Defaulting to same space for all atoms.
  warnings.warn("Space provided as string. Defaulting to same space for all atoms.", UserWarning)
c:\Users\Ivan_lg\Desktop\Doctorado\SegundoAnio\TFGMariadelMar\Calculos\hartreeFock\information_measures.py:589: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  integral, error = integrate.quad(integrando, 0, np.inf, epsabs=epsabs, epsrel=epsrel, limit=1000)
c:\Users\Ivan_lg\Desktop\Doctorado\SegundoAnio\TFGMariadelMar\Calculos\hartreeFock\information_measures.py:595: RuntimeWarning: invalid value encountered in log
  'RenyiEntropy': (1/(a-1))*np.log(integral),
c:\Users\Ivan_lg\Desktop\Doctorado\SegundoAnio\TFGMariadelMar\Calculos\hartreeFock\information_measures.py:589: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolatio